# Mecklenburg-Vorpommern

Every federal state is represented by its own input directory and is processed into a NUTS level 2 directory containing a sub-folder for each discharge location. These folder names are derived from NUTS and reflect the CAMELS id. The NUTS level 2 code for Mecklenburg-Vorpommern is `DE8`.

To pre-process the data, you need to write (at least) two functions. One should extract all metadata and condense it into a single `pandas.DataFrame`. This is used to build the folder structure and derive the ids.
The second function has to take an id, as provided by the state authorities, called `provider_id` and return a `pandas.DataFrame` with the transformed data. The dataframe needs the three columns `['date', 'q' | 'w', 'flag']`.

For easier and unified output handling, the `camelsp` package contains a context object called `Bundesland`. It takes a number of names and abbreviations to identify the correct federal state and returns an object that holds helper and save functions.

The context saves files as needed and can easily be changed to save files with different strategies, ie. fill missing data with NaN, merge data into a single file, create files for each variable or pack everything together into a netcdf.

In [ ]:
import pandas as pd
import numpy as np
from pandas.errors import ParserError
import os
from pprint import pprint
from tqdm import tqdm
from typing import Union, Dict
import zipfile
from datetime import datetime as dt
from dateparser import parse
import warnings

from camelsp import Bundesland

The context can also be instantiated as any regular Python class, ie. to load only the default input data path, that we will user later.

In [ ]:
# the context also makes the input path available, if camelsp was install locally
BASE = Bundesland('Meckpom').input_path
BASE

First extract the ZIP:

In [ ]:
# extract the ZIP in place
if not os.path.exists(os.path.join(BASE, 'MetaDaten.csv')):
    with zipfile.ZipFile(os.path.join(BASE, 'w_q.zip')) as z:
        for f in z.filelist:
            z.extract(f, BASE)

### Metadata reader

Define the function that extracts / reads and eventually merges all metadata for this federal state. You can develop the function here, without using the Bundesland context and then later use the context to pass extracted metadata. The Context has a function for saving *raw* metadata, that takes a `pandas.DataFrame` and needs you to identify the id column.
Here, *raw* refers to provider metadata, that has not yet been transformed into the CAMELS-de Metadata schema.

In [ ]:
# define the function 
def read_meta(base_path) -> pd.DataFrame:
    path = os.path.join(base_path, 'MetaDaten.csv')
    meta = pd.read_csv(path, encoding='latin1', sep=";")
    return meta

# test it here
metadata = read_meta(BASE)

metadata

In [ ]:
# the id column will be pegelkennzahl
id_column = 'pegelkennzahl'

## file extract and parse

Similar to Niedersachsen, this data should again involve some substantial pivoting. 

In [ ]:
raw = pd.read_csv(os.path.join(BASE, 'gesamter Datensatz.csv'), encoding='latin1', sep=";", parse_dates=['datum'])
raw

Ok, other than in Niedersachen, we have different parameter and types here. Maybe filter for Tagesmittel and split by q and w first.

In [ ]:
print(f"Parameter: {raw.parameter.unique()}")
print(f"Variabeln: {raw.typ.unique()}")

First, filter for `'Tagesmittel'` only, then split py `'parameter'` and copy into two new DataFrames. This should make stuff bit easier.

In [ ]:
# split and group the data into Tagesmittel of q and w data
for par, df in raw.where(raw.typ == 'Tagesmittel').dropna(axis=0).groupby('parameter'):
    if par == 'Wasserstand (PNP)':
        w = df.copy()
    elif par == 'Durchfluss':
        q = df.copy()

print(f"Q: {len(q)}   W: {len(w)}")

Finally write a function that extracts the data from the Dataframes and returns empty DataFrames if the pkz cannot be found in the large df.

In [ ]:
def extract_file(pkz: Union[int, str], variable: str, store_df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts the variable for the given pkz from the large storage dataframe.
    Returns an empty dataframe if the pkz was not found.
    """
    # always use string pkz
    pkz = str(pkz)

    # filter
    df = store_df.where(store_df.pkz.astype(str) == pkz).dropna(axis=0)

    # check if we found something
    if df.empty:
        return pd.DataFrame(columns=['date', variable.lower(), 'flag'])
    else:
        return pd.DataFrame({
            'date': df.datum,
            variable.lower(): df.messwert,
            'flag': np.NaN
        })

test_df = extract_file('59910.5', 'w', w)
test_df

### Finally run

Now, the Q and W data can be extracted. The cool thing is, that all the id creation, data creation, merging and the mapping from our ids to the original ids and files is done by the context. This is helpful, as we less likely screw something up.

In [ ]:
with Bundesland('Mecklenburg-Vorpommern') as bl:
    # save the metadata
    bl.save_raw_metadata(metadata, id_column, overwrite=True)

    # for reference, call the nuts-mapping as table
    nuts_map = bl.nuts_table
    print(nuts_map.head())

    
    with warnings.catch_warnings(record=True) as warns:
        for provider_id in tqdm(metadata[id_column].values.astype(str)):
            # get q and w
            q_df = extract_file(provider_id, 'q', q)
            w_df = extract_file(provider_id, 'w', w)

            bl.save_timeseries(q_df, provider_id)
            bl.save_timeseries(w_df, provider_id)

        # check if there were warnings (there are warnings)
        if len(warns) > 0:
            log_path = bl.save_warnings(warns)
            print(f"There were warnings during the processing. The log can be found at: {log_path}")
